In [2]:
# 1. creation d'une session spark


# 2. Creation d'un RDD à partir d'un fichier text

# 3.a affichage du RDD

# 3.b affichage des données du RDD => operation de recuperation des données

# 4. Afficher le nombre de lignes dans le fichier

# 5. Afficher les 10 premières lignes du fichier

# 6. Filtrer les lignes qui contiennent le mot "Lewis"

# 7. Transformer les lignes qui contiennent lewis en majuscule
# cas 1 : à partir du debut

# cas 2 : en réutilisant un RDD cree precedemment

# 8. Compter le nombre d'occurences du mot "Wonderland" dans chaque ligne


===== affichage du RDD =====
work/DataSource/test.txt MapPartitionsRDD[8] at textFile at NativeMethodAccessorImpl.java:0
['Project Gutenberg’s', 'Alice’s Adventures in Wonderland', 'by Lewis Carroll', 'This eBook is for the use', 'of anyone anywhere', 'at no cost and with', 'Alice’s Adventures in Wonderland', 'by Lewis Carroll', 'This eBook is for the use', 'of anyone anywhere', 'at no cost and with', 'This eBook is for the use', 'of anyone anywhere', 'at no cost and with', 'Project Gutenberg’s', 'Alice’s Adventures in Wonderland', 'by Lewis Carroll', 'This eBook is for the use', 'of anyone anywhere', 'at no cost and with', 'Alice’s Adventures in Wonderland', 'by Lewis Carroll', 'This eBook is for the use', 'of anyone anywhere', 'at no cost and with', 'This eBook is for the use', 'of anyone anywhere', 'at no cost and with', 'Project Gutenberg’s', 'Alice’s Adventures in Wonderland', 'by Lewis Carroll', 'This eBook is for the use', 'of anyone anywhere', 'at no cost and with', 'Alice’s Ad